In [ ]:
#We read the semantic representation from a trained network layer. 
#We choose ResNet-18 model (trained on the ImageNet dataset with 1000 categories and 1 million images).

In [8]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import pandas as pd
import os
from scipy.misc import imread, imsave, imresize
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import kendalltau

In [3]:
def computeRepresentationTensor(imageName,model,layer):
    
    """Compute the tensor representation for the image receive as input """
    
    
    img = Image.open(imageName)
    
    #We prepare our image to be like the images ResNet 18 model was trained on.
    #For this we scale it, normalize it and convert it to a tensor
    img = img.convert("RGB")
    preprocess=transforms.Compose([   transforms.Resize((224, 224)),
                                      transforms.ToTensor(), 
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
                                     ]
                                 )
    
    #all above transformations apllied to the input image
    #unsqueeze=>reshape the image from (3, 224, 224) to (1, 3, 224, 224). 
    #PyTorch expects a 4-dimensional input with the first dimension = to the number of samples.
    imgTransformed = preprocess(img).unsqueeze(0)
    
    #Where we copy the avearge pool layer representation
    features = []

    # Define a function that will copy the output of a layer (see the above explanation for the parameters used)
    def hook(m, i, o):
        features.extend(o.view(o.size(0),-1).cpu().data)

    #Attach the hook to our selected layer (average pool layer in this case)
    h = layer.register_forward_hook(hook)
    
    #Run the model on the input image
    model(imgTransformed )

    #Detach the hook
    h.remove()
    plt.imgshow(features)
    
    # return the image representation on the selected layer
    return features[0]
    
    

In [6]:
def calc_sim():
    df = pd.DataFrame(columns=["word1", "word2", "SL","ESL", "model"])
    data = pd.read_excel("Est-SimLex999.xlsx")
    data = data[(data["conc(w1)"] >=4.8) & (data["conc(w2)"]>=4.8)]
    print(data.shape)
    averages = []
    simlex = []
    rerate = []
    model = models.resnet18(pretrained=True)
    
    #look at the model
    #print (model)
    
    # We select the layer we interested in, the average pool layer
    layer = model._modules.get('avgpool')
    
    #check we've selected the right layer
    #print (layer)
    
    #We must eliminate dropout during testing
    model.eval()
    for i, row in data.iterrows():
        word1 = row["word 1"].strip()
        word2 = row["word 2"].strip()
        w1_images = os.listdir("F:\\db\\"+word1)
        w2_images = os.listdir("F:\\db\\"+word2)
        similarities = []
        features = [computeRepresentationTensor("F:\\db\\"+word2 + "\\"+ w2, model , layer) for w2 in w2_images]
        for w1 in w1_images:
            try:
                features1 = computeRepresentationTensor("F:\\db\\"+word1 + "\\"+w1, model, layer)
                for  features2 in features:  
                    cos = nn.CosineSimilarity(0)
                    cosineSimValue = cos (features1, features2)
                    similarities.append(cosineSimValue.item())
            except Exception as e:
                print(e)

        averageSimilarity = sum(similarities)/len(similarities)
        df = df.append({"word1":word1, "word2":word2, "SL":row["SimLex999"], "ESL":row["Average"], "model":averageSimilarity}, 
                      ignore_index=True)

        
    df.to_excel("results_from_resnet.xlsx")
    
        

In [ ]:
calc_sim()